## logistic regression model

#### import libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#### load cleaned feature data table

In [2]:
df = pd.read_feather('../data/processed/df_to_model_labs')

In [3]:
df.columns

Index(['time_on_vent', 'anchor_age', 'spontrr', 'heartrate', 'std_spontrr',
       'weight', 'sodium', 'abg_po2', 'abg_ph', 'hco3', 'abg_pco2',
       'bloodpressure', 'std_pulseox', 'std_heartrate', 'creatinine', 'bun',
       'height', 'lactic_acid', 'hemoglobin', 'wbg', 'tidalvolume', 'temp',
       'std_bloodpressure', 'pulseox', 're_intub_class', 'gender',
       'admission_type'],
      dtype='object')

In [4]:
df[df['re_intub_class']==0]

,time_on_vent,anchor_age,spontrr,heartrate,std_spontrr,weight,sodium,abg_po2,abg_ph,hco3,...,lactic_acid,hemoglobin,wbg,tidalvolume,temp,std_bloodpressure,pulseox,re_intub_class,gender,admission_type
0,3.257455,5184.0,19.0,79.0,1.551546,123.2,139.0,267.0,7.55,22.0,...,8.1,9.2,12.2,387.0,36.444444,3.060937,100.0,0,F,EW EMER.
1,2.833213,529.0,33.0,122.0,1.521199,264.0,132.0,162.0,7.41,23.0,...,0.9,12.2,8.3,527.0,39.166667,1.994816,96.0,0,M,EW EMER.
2,6.363430,6561.0,24.0,63.0,1.025631,198.0,140.0,77.0,7.48,26.0,...,1.0,9.4,16.0,437.0,36.833333,2.004007,96.0,0,F,EW EMER.
3,4.429228,3721.0,18.0,73.0,2.246625,199.5,139.0,107.0,7.39,26.0,...,1.2,8.4,24.3,556.0,36.444444,2.124247,100.0,0,F,EW EMER.
4,5.077359,3844.0,33.0,96.0,1.940565,136.8,140.0,298.0,7.48,22.0,...,1.7,9.5,11.8,417.0,36.944444,1.160300,100.0,0,F,EW EMER.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5477,2.740840,3249.0,19.0,86.0,1.230260,147.4,137.0,122.0,7.39,25.0,...,2.0,11.2,16.2,422.0,36.888889,0.922899,100.0,0,F,DIRECT EMER.
5478,1.811017,7396.0,18.0,80.0,1.623960,113.5,135.0,126.0,7.35,21.0,...,2.1,7.5,6.5,406.0,36.388889,2.730563,100.0,0,M,OBSERVATION ADMIT
5479,4.964824,3481.0,21.0,88.0,1.110344,233.0,139.0,140.0,7.36,21.0,...,1.3,8.0,12.9,334.0,36.555556,2.446675,96.0,0,M,EW EMER.
5480,3.054787,3249.0,19.0,83.0,1.641582,255.2,142.0,134.0,7.37,23.0,...,1.7,10.6,15.4,401.0,37.333333,1.482939,96.0,0,F,ELECTIVE


## Handle categoricals

In [5]:
X = df[df.columns.drop('re_intub_class')]
y = df['re_intub_class']

In [6]:
# Import libraries 
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [7]:
numeric_features  = df[df.columns.drop(['gender','admission_type','re_intub_class'])].columns
numeric_transformer = Pipeline(steps=[
   # ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = df[['gender','admission_type']].columns
categorical_transformer = Pipeline(steps=[
   # ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])


# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('smote', SMOTE(k_neighbors = 5,random_state=101)),
                      ('classifier', LogisticRegression())])

param_grid = [
    {'classifier' : [LogisticRegression()],
     'classifier__penalty' : ['l1', 'l2'],
    'classifier__C' : np.logspace(-4, 4, 20),
    'classifier__solver' : ['liblinear']},
    {'classifier' : [RandomForestClassifier()],
    'classifier__n_estimators' : list(range(10,101,10)),
    'classifier__max_features' : list(range(6,32,5))}
]

svc_param_grid ={ 'svc__kernel':('linear', 'rbf'), 
                 'svc__C': [0.1,1, 10, 100, 1000], 
                 'svc__gamma': [1,0.1,0.01,0.001,0.0001]}

gs = GridSearchCV(clf, param_grid, verbose=True, return_train_score=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state = 101)

clf.fit(X_train, y_train)
#print("model score: %.3f" % clf.score(X_test, y_test))

predictions = clf.predict(X_test)
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

              precision    recall  f1-score   support

           0       0.93      0.61      0.73      1107
           1       0.13      0.56      0.21       114

    accuracy                           0.60      1221
   macro avg       0.53      0.58      0.47      1221
weighted avg       0.86      0.60      0.69      1221

[[671 436]
 [ 50  64]]


In [8]:
clf.predict_log_proba(X_test)

array([[-0.63880362, -0.75061453],
       [-0.36545086, -1.1837903 ],
       [-0.40754412, -1.09446718],
       ...,
       [-0.80011856, -0.59652095],
       [-1.4222081 , -0.27599184],
       [-0.1553951 , -1.93847598]])

#### 5. Evaulate model

In [9]:
from sklearn.metrics import classification_report, confusion_matrix

In [10]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.93      0.61      0.73      1107
           1       0.13      0.56      0.21       114

    accuracy                           0.60      1221
   macro avg       0.53      0.58      0.47      1221
weighted avg       0.86      0.60      0.69      1221



In [11]:
print(confusion_matrix(y_test,predictions))

[[671 436]
 [ 50  64]]


In [12]:
import joblib

In [13]:
#joblib.dump(logmodel, "reintubate_model_log.sav")

In [14]:
#joblib.dump(scaler, "reintubate_scaler.sav")

## Train model

### perform train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#### 3. SMOTE IT!

from collections import Counter
counter = Counter(y_train)
print(counter)

# transform the dataset
oversample = SMOTE(random_state = 101)

X_smote, y_smote = oversample.fit_resample(X_train, y_train)

counter = Counter(y_smote)
print(counter)

#### 4. Do logistic regression model

from sklearn.linear_model import LogisticRegression

logmodel = LogisticRegression(max_iter=1000, C=0.0001)
logmodel.fit(X_smote,y_smote)

import pickle

In [15]:
# Save the model as a pickle in a file 
#pickle.dump(logmodel, open("reintubate_model_log", 'wb')) 

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_features  = df[df.columns.drop(['gender','admission_type','re_intub_class'])].columns
#numeric_transformer = ('scaler', StandardScaler())
numeric_transformer = make_column_transformer(
    (StandardScaler(), numeric_features),
    remainder='passthrough')


categorical_features = df[['gender','admission_type']].columns
#categorical_transformer =  ('onehot', OneHotEncoder(drop='first'))
categorical_transformer = make_column_transformer(
    (OneHotEncoder(drop='first'), categorical_features),
    remainder='passthrough')

#preprocessor = make_column_transformer(
 #   transformers=[
  #      ('num', numeric_transformer, numeric_features),
   #     ('cat', categorical_transformer, categorical_features)],
#remainder ='passthrough')

clf = Pipeline(steps=[('num', numeric_transformer),
                      ('cat', categorical_transformer),
                      ('classifier', LogisticRegression())])

#### 2. Perform feature scaling

Because the range of values in the features are not necessarily in the same order of magnitude, we will scale the feature data prior to training the model.

* actually... they might not be far off! 

mask=['spontRR', 'stdABP', 'meanABP', 'stdSpontRR', 'pulseox', 'stdPulseox',
       'temp', 'heartRate', 'stdHeartRate', 'weight', 'height', 'anchor_age',
       'time_on_vent']

X_traina = X_train.copy()
X_testa = X_test.copy()

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train.loc[:,mask])
X_traina.loc[:,mask] = scaler.transform(X_train.loc[:,mask])

#X_train = scaler.transform(X_train)
X_testa.loc[:,mask] = scaler.transform(X_test.loc[:,mask])